# Elo Bidimensional aplicado no Tênis

Esse notebook foi elaborado de forma a servir como uma espécie de "diário" do processo da modelgem de resultados de tênis utilizando a ideia do Elo Bidimensional.

O mesmo trás funções que buscam fazer a estimativa das proficiências no tipo $v = [v_1, ~v_2]$ para jogadores de tênis, bem como executar o cálculo da probabilidade de um atleta de proficiência $v = [v_1, ~v_2]$ ganhar de um atleta com proficiência $u = [u_1, ~u_2]$.

### Primeira Formulação

Abaixo temos as funções elaboradas para calcular a probabilidade e, dada tal probabilidade, calcular os pares u e v de proficiência dos atletas. Por enquanto, funciona apenas para o caso em que temos dois atletas.

In [1]:
import math

def calcula_prob(v, u):
    # recebe dois vetores v = [v1, v2] e u = [u1, u2]
    # retorna um vetor p = [p1, p2] com a prob do jogador 1 ganhar do 2 e do 2 ganhar do 1
    p = [0, 0]
    p[0] = math.exp(u[0]*v[1])/(math.exp(u[0]*v[1]) + math.exp(u[1]*v[0]))
    p[1] = 1 - p[0]
    
    return p

def gera_parametro(p):
    # p = [p0, p1], onde p0 = x/(x + y) e p1 = y/(x + y), com x = e^(u0*v1) e y = e^(v0*u1)
    # queremos encontrar v = [v0, v1] e u = [u0, u1]
    x = math.log(p[0]) # faz x = ln(x) - ln(x + y) = ln(x) = u0*v1
    y = math.log(p[1]) # faz y = ln(y) - ln(x + y) = ln(y) = v0*u1
    
    # considerando u0 = 1 = v0, temos que 
    u = [1, x]
    v = [1, y]
    
    '''
    tentativa de normalizar
    
    v = [1/math.sqrt(1 + x**2), x/math.sqrt(1 + x**2)]
    u = [1/math.sqrt(1 + y**2), y/math.sqrt(1 + y**2)]
    '''
    
    return [u, v]

Abaixo definimos 3 proficiências com vetores que formam ângulos de $120^{\circ}$ entre si. Note que a probabilidade de $v_2$ ganhar de $v_1$ é maior que a de $v_2$ perder para o mesmo. Além disso, a probabilidade de $v_3$ ganhar de $v_2$ é maior que a $v_3$ perder, mas a probabilidade de $v_3$ perder para $v_1$ é maior que a de ganhar de $v_1$, assim, não há linearidade no modelo.

In [2]:
v1 = [   1,               0]
v2 = [-0.5,  math.sqrt(3)/2]
v3 = [-0.5, -math.sqrt(3)/2]

print("Jogo entre v1 e v2:", calcula_prob(v1, v2))
print("Jogo entre v2 e v3:", calcula_prob(v2, v3))
print("Jogo entre v3 e v1:", calcula_prob(v3, v1))

Jogo entre v1 e v2: [0.2960820052793571, 0.7039179947206429]
Jogo entre v2 e v3: [0.2960820052793571, 0.7039179947206429]
Jogo entre v3 e v1: [0.2960820052793571, 0.7039179947206429]


Abaixo temos o cálculo da probabilidade com as proficiências $v_1$ e $v_2$, em seguida, tomamos o vetor de probabilidades e calculamos os parâmetros, os quais diferem dos valores iniciais ($v_1$ e $v_2$), mas que, como podemos ver em seguida, geram as mesmas probabilidades.

In [3]:
p = calcula_prob(v1, v2)
param = gera_parametro(p)
q = calcula_prob(param[0], param[1])
print("Probabilidades com p:", p)
print("Parâmetros:", param)
print("Probabilidades os parâmetros encontrados:", q)

Probabilidades com p: [0.2960820052793571, 0.7039179947206429]
Parâmetros: [[1, -1.2171188181652253], [1, -0.3510934143807867]]
Probabilidades os parâmetros encontrados: [0.2960820052793571, 0.7039179947206429]


Agora a ideia é buscar fazer a mesma estimação dos parâmetros, mas dando como entrada alguns pares ordenados que representam as probabilidades (ou uma estimativa das mesmas) e com isso gerar os parâmetros.

Após conversar com os professores Moacyr e Paulo Cezar, foi proposta a modelagem por meio de uma outra função para a probabilidade, como podemos ver abaixo.

### Nova Formulação

Abaixo temos o desenvolvimento das funções que calculam as propabilidades, onde $P(A\mbox{ ganhar de }B) = \dfrac{1}{1 + \exp(-logit)}$, onde $logit = \dfrac{v_2}{u_1} - \dfrac{u_2}{v_1}$, além do desenvolvimento da função que calcula o log da verossimilhança negativa, dessa teremos valores positivos, uma vez que o log de cada termo do produtório é negativo, e buscaremos minimizar esse valor.

In [4]:
import math

def calcula_prob(v, u):
    # calcula a probabilidade de acordo com a ideia do logit
    # recebe dois vetores v = [v1, v2] e u = [u1, u2]
    # retorna um vetor p = [p1, p2] com a prob do jogador 1 ganhar do 2 e do 2 ganhar do 1
    logit = v[1]/u[0] - u[1]/v[0]
    p = [0, 0]
    p[0] = 1/(1 + math.exp(-logit))
    p[1] = 1 - p[0]
    
    return p

def verossimilhanca(resultados, jogadores):
    # faz o cálculo da verossimilhança negativa dado como entrada uma lista
    # de resultados e uma lista de parâmetros, conforme abaixo
    
    # resultados é uma lista de lista/matriz n x 2, onde cada lista/linha
    # representa um jogo, sendo da forma [ID_w, ID_l]
    # jogadores é a lista de parâmetros dos jogadores, assim parâmetros[0]
    # contém os parâmetros do jogador de ID = 0
    
    log_ver = 0
    p = [0, 0]
    
    for i in range(len(resultados)):
        p = calcula_prob(jogadores[resultados[i][0]], jogadores[resultados[i][1]])
        log_ver += math.log(p[0])
    
    return - log_ver

Definida as funções, abaixo temos um pequeno teste para ver como a função que calcula as probabilidades está funcionando:

In [5]:
P1 = [   1,               0]
P2 = [-0.5,  math.sqrt(3)/2]
P3 = [-0.5, -math.sqrt(3)/2]

print("Jogo entre P1 e P2:", calcula_prob(P1, P2))
print("Jogo entre P2 e P3:", calcula_prob(P2, P3))
print("Jogo entre P3 e P1:", calcula_prob(P3, P1))

Jogo entre P1 e P2: [0.2960820052793571, 0.7039179947206429]
Jogo entre P2 e P3: [0.030351090329424395, 0.9696489096705756]
Jogo entre P3 e P1: [0.2960820052793571, 0.7039179947206429]


Estranhei o fato de que na abordagem anterior, vista abaixo, esses 3 vetores geravam probabilidades iguais para esses jogos, ou seja, tínhamos $P(P_1\mbox{ vencer }P_2) = (P_2\mbox{ vencer }P_3) = (P_3\mbox{ vencer }P_1)$, o que agora difere.

Abaixo temos um teste da função que calcula o log da verossimilhança negativa:

In [6]:
resultados = [[0, 1], [1, 2], [2, 0], [2, 1]]
jogadores = [P1, P2, P3]

verossimilhanca(resultados, jogadores)

5.959981695062614

O primeiro elemento da lista de resultados indica que o jogador 0 (P1), ganhou do jogador 1 (P2). O mesmo sendo válido aos demais elementos dessa lista.

Tendo elaborado a função de verossimilhança, devemos agora buscar os parâmetros dos jogadores que irão minimizar a mesma (lembrando que a função retorna o oposto do log da verossimilhança). Para isso, estou utilizando a função minimize, da biblioteca SciPy, cuja documentação pode ser vista aqui: https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html

Pela documentação, percebi que a função a ser minimizada deve ter um certo padrão em seus parâmetros, sendo que a primeira entrada da função são as variáveis que estamos buscando e as demais são parâmetros fixos. Além disso, a entrada que contém as variáveis deve ser um vetor de uma idimensão, então fiz algumas alterações nas funções elaboradas acima para que respeite essa chamada da função:

In [7]:
def calcula_prob(v, u):
    # calcula a probabilidade de acordo com a ideia do logit
    # recebe dois vetores v = [v1, v2] e u = [u1, u2]
    # retorna p, a prob do jogador 1 ganhar do jogador 2
    logit = v[1]/u[0] - u[1]/v[0]
    
    # coloquei esse try pois tem vezes que essa exponencial resulta em um valor muito grande
    # gerando o OverflowError
    try:
        p = 1/(1 + math.exp(-logit))
    except OverflowError:
        p = float(1e-31)
    
    return p

def verossimilhanca(jogadores, resultados):
    # faz o cálculo da verossimilhança negativa dado como entrada uma lista
    # de resultados e uma lista de parâmetros, conforme abaixo
    
    # jogadores é a lista de parâmetros dos jogadores. Nela os parâmetros
    # do jogador de ID = i estão em parâmetros[2*i] e em parâmetros[2*i + 1]
    # resultados é uma lista de lista/matriz n x 2, onde cada lista/linha
    # representa um jogo, sendo da forma [ID_w, ID_l]
    
    log_ver = 0
    
    for i in range(len(resultados)):
        indice_1 = 2*resultados[i][0]
        indice_2 = 2*resultados[i][1]
        param_1 = [jogadores[indice_1], jogadores[indice_1 + 1]]
        param_2 = [jogadores[indice_2], jogadores[indice_2 + 1]]
        p = calcula_prob(param_1, param_2)
        log_ver += math.log(p)
    
    return - log_ver

Feita tais alterações nas funções, devemos alterar as entradas da função para ficar conforme a função que temos nesse momento.

In [8]:
resultados = [[0, 1], [1, 2], [2, 0], [2, 1]]
jogadores = [1, 0, -0.5, math.sqrt(3)/2, -0.5, -math.sqrt(3)/2]

verossimilhanca(jogadores, resultados)

5.959981695062614

Agora podemos utilizar o minimizador, o qual buscará o vetor jogadores que minimiza a função de verossimilhança. Como temos a restrição de que o primeiro parâmetro de cada jogador é positivo, vou utilizar o método SLSQP para encontrar os parâmetros que minimizam a verossimilhança. Para isso, vamos primeiramente definir quais são as restrições:

In [9]:
cons = ({'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[0]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[2]},
        {'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[4]})

Ainda preciso encontrar uma forma mais esperta para fazer essas restrições, pois fazer na mão para 3 atletas é simples, mas para um grande número de atletas fica complicado.

Definida tais restrições, vamos minimizar a função:

In [10]:
from scipy.optimize import minimize

results = minimize(verossimilhanca, jogadores, args = resultados, method = 'SLSQP', constraints = cons)

Normalizando o vetor jogadores de forma que o menor valor, em módulo, seja igual a $1$, temos:

In [11]:
results.x = results.x * (1/min(abs(results.x)))
print("Os parâmetros encontrados são:\n", results.x)

Os parâmetros encontrados são:
 [505.70472945 251.77736851  40.77817833 -42.68048805 551.18195062
   1.        ]


Olhando para a verossimilhança gerada pelos mesmos, temos:

In [12]:
results.fun

2.3369976685748237

Fazendo outro teste, agora com outro vetor inicial e construindo a restrição com um loop for, temos:

In [13]:
cons = []
jogadores = []
for i in range(3):
    cons.append({'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[2*i]})
    jogadores.append(1)
    jogadores.append(0)

In [14]:
cons = tuple(cons)

In [15]:
results = minimize(verossimilhanca, jogadores, args = resultados, method = 'SLSQP', constraints = cons)

In [16]:
results.x

array([ 2.99009532e-06,  1.69032020e-07,  1.67042410e+00, -3.08439143e-01,
        1.67042279e+00,  3.08440896e-01])

In [17]:
results.fun

1.4201971731668042

Note que o mínimo encontrado difere do mínimo encontrado anteriormente.

#### Testando com os dados obtidos

Primeiramente, vou definir algumas funções para facilitar na definição dos parâmetros do minimizador:

In [22]:
from openpyxl import load_workbook
import numpy as np

def pega_jogos(file):
    games = load_workbook(file)

    jogos = games['Planilha1']

    resultados = []

    for row in jogos.iter_rows():
        jogo = []
        for cell in row:
            if cell.value != 'w' and cell.value != 'l':
                jogo.append(cell.value)
        resultados.append(jogo)
    
    if resultados[0] == []:
        resultados.pop(0)
    return resultados

def conta_jogadores(resultados):
    contagem = 0
    for i in range(len(resultados)):
        for j in resultados[i]:
            if j > contagem:
                contagem = j
    
    return contagem + 1

def cria_restricoes(contagem):
    restricoes = []
    for i in range(contagem):
        restricoes.append({'type': 'ineq', 'fun': lambda par_jogadores: par_jogadores[2*i]})
    
    restricoes = tuple(restricoes)
    return restricoes

def cria_jogadores(contagem):
    jogadores = []
    for i in range(contagem):
        jogadores.append(np.random.random())
        jogadores.append(np.random.random())
    
    return jogadores

def calcula_parametros(file):
    resultados = pega_jogos(file)
    contagem = conta_jogadores(resultados)
    cons = cria_restricoes(contagem)
    jogadores = cria_jogadores(contagem)
    results = minimize(verossimilhanca, jogadores, args = resultados, method = 'SLSQP', constraints = cons)
    
    return results

Agora, vou utilizar a função calcula_parametros para gerar os parâmetros dos jogadores:

In [28]:
results = calcula_parametros('matches.xlsx')

Onde obtemos o seguinte valor de mínimo:

In [29]:
results.fun

20300.864333136815